<a href="https://colab.research.google.com/github/wiamfa/Speech_emotion_recognition/blob/main/Wiam/wav2vec2-large-robust-ft-libri-960h-finetuned-ravdess-v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
model_checkpoint = "facebook/wav2vec2-large-robust-ft-libri-960h"
batch_size = 16
ravdess_emotions = ["neutral", "calm", "happy", "sad", "angry", "fearful", "disgust", "surprised"]



In [2]:
%%capture
!pip install datasets
!pip install transformers
!pip install librosa

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
%%capture
!apt install git-lfs

In [5]:
from transformers.utils import send_example_telemetry

send_example_telemetry("audio_classification_notebook", framework="pytorch")

#test HuggingFace on Hubert

In [6]:
from datasets import load_dataset, load_metric, Audio

In [7]:
dataset = load_dataset("xbgoose/ravdess",split="train")
metric = load_metric("accuracy")

<ipython-input-7-0b02b7a0bda2>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [8]:
dataset000=dataset.remove_columns([ 'modality', 'vocal_channel', 'emotional_intensity', 'statement', 'repetition', 'actor', 'gender'])

In [9]:
from datasets.tasks.text_classification import ClassLabel
#resampling to 16Khz
dataset00 = dataset000.cast_column("audio", Audio(sampling_rate=16000))
#Label identification
dataset0 = dataset00.cast_column("emotion", ClassLabel(num_classes=8, names=ravdess_emotions))


In [10]:
dataset1=dataset0.train_test_split(test_size=0.2)

In [11]:
dataset1

DatasetDict({
    train: Dataset({
        features: ['audio', 'emotion'],
        num_rows: 1152
    })
    test: Dataset({
        features: ['audio', 'emotion'],
        num_rows: 288
    })
})

In [12]:
dataset1["test"][100]

{'audio': {'path': '03-01-04-01-02-01-20.wav',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -3.18358673e-11,  2.20410357e-11, -1.49462664e-11]),
  'sampling_rate': 16000},
 'emotion': 3}

In [13]:
dataset1["train"].features

{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'emotion': ClassLabel(names=['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised'], id=None)}

In [14]:
labels = dataset1["train"].features["emotion"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["7"]

'surprised'

In [15]:
import random
from IPython.display import Audio, display

for _ in range(5):
    rand_idx = random.randint(0, len(dataset1["train"])-1)
    example = dataset1["train"][rand_idx]
    audio = example["audio"]

    print(f'Label: {id2label[str(example["emotion"])]}')
    print(f'Shape: {audio["array"].shape}, sampling rate: {audio["sampling_rate"]}')
    display(Audio(audio["array"], rate=audio["sampling_rate"]))
    print()

Label: disgust
Shape: (62997,), sampling rate: 16000



Label: disgust
Shape: (76343,), sampling rate: 16000



Label: happy
Shape: (59259,), sampling rate: 16000



Label: angry
Shape: (61395,), sampling rate: 16000



Label: fearful
Shape: (56056,), sampling rate: 16000


In [16]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [17]:
max_duration = 2.0  # seconds

In [18]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
    )
    return inputs

In [19]:
preprocess_function(dataset1['train'][:5])

{'input_values': [array([ 2.0146843e-05,  2.0146788e-05,  2.0146788e-05, ...,
       -3.5561718e-02, -2.2673756e-01, -1.5317419e-01], dtype=float32), array([ 1.2844319e-04,  1.2844319e-04,  1.2844319e-04, ...,
       -2.6296300e-01, -3.7636468e-01, -3.2878077e-01], dtype=float32), array([-1.5046472e-04, -1.4319735e-04, -1.5153008e-04, ...,
       -4.3786889e-01, -4.6989697e-01, -4.6022704e-01], dtype=float32), array([ 8.0212390e-05,  8.0212390e-05,  8.0212390e-05, ...,
       -4.0530186e+00, -4.8032060e+00, -4.7237101e+00], dtype=float32), array([ 0.00470408,  0.00483302,  0.00471952, ...,  0.05432959,
       -0.11964495, -0.29945967], dtype=float32)], 'attention_mask': [array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([1, 1, 1, ..., 1, 1, 1], dtype=int32)]}

In [20]:
encoded_dataset = dataset1.map(preprocess_function, remove_columns=["audio"], batched=True)
encoded_dataset

Map:   0%|          | 0/1152 [00:00<?, ? examples/s]

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['emotion', 'input_values', 'attention_mask'],
        num_rows: 1152
    })
    test: Dataset({
        features: ['emotion', 'input_values', 'attention_mask'],
        num_rows: 288
    })
})

In [21]:
encoded_dataset["train"][0]["emotion"]

6

In [22]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-large-robust-ft-libri-960h and are newly initialized: ['classifier.bias', 'projector.weight', 'projector.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
!pip uninstall accelerate

Found existing installation: accelerate 0.21.0
Uninstalling accelerate-0.21.0:
  Would remove:
    /usr/local/bin/accelerate
    /usr/local/bin/accelerate-config
    /usr/local/bin/accelerate-launch
    /usr/local/lib/python3.10/dist-packages/accelerate-0.21.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/accelerate/*
Proceed (Y/n)? Y
  Successfully uninstalled accelerate-0.21.0


In [24]:
!pip install accelerate -U

  Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)


In [25]:
!accelerate config



2023-08-19 13:00:45.433611: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
In which compute environment are you running?
Please input a choice index (starting from 0), and press enter
 ➔  This machine
    AWS (Amazon SageMaker)

This machine
Which type of machine are you using?
Please input a choice index (starting from 0), and press enter
 ➔  No distributed training
    multi-CPU
    multi-XPU
    multi-GPU
    multi-NPU
    TPU

No distributed training
Do you want to run your training on CPU only (even if a GPU / Apple Silicon device is available)? [yes/NO]:NO
Do you wish to optimize your script with torch dynamo?[yes/NO]:
Do you want to use DeepSpeed? [yes/NO]: 
What GPU(s) (by id) should be used for training on this machine as a comma-seperated list? [all]:
Do you wish to use FP16 or BF16 (mixed precision)?
Please input a choice index (starting from 0), and press enter
 ➔  no
    fp16
    bf16
    fp8
1
fp16
accelerate configuration 

In [26]:
!accelerate env

2023-08-19 13:01:09.672341: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Copy-and-paste the text below in your GitHub issue

- `Accelerate` version: 0.21.0
- Platform: Linux-5.15.109+-x86_64-with-glibc2.35
- Python version: 3.10.12
- Numpy version: 1.23.5
- PyTorch version (GPU?): 2.0.1+cu118 (True)
- PyTorch XPU available: False
- PyTorch NPU available: False
- System RAM: 51.00 GB
- GPU type: Tesla V100-SXM2-16GB
- `Accelerate` default config:
	- compute_environment: LOCAL_MACHINE
	- distributed_type: NO
	- mixed_precision: fp16
	- use_cpu: False
	- num_processes: 1
	- machine_rank: 0
	- num_machines: 1
	- gpu_ids: all
	- rdzv_backend: static
	- same_network: True
	- main_training_function: main
	- downcast_bf16: no
	- tpu_use_cluster: False
	- tpu_use_sudo: False
	- tpu_env: []


In [27]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-ravdess-v3",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    gradient_checkpointing=True,


)

In [28]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [29]:
encoded_dataset["train"]

Dataset({
    features: ['emotion', 'input_values', 'attention_mask'],
    num_rows: 1152
})

In [30]:
encoded_dataset1= encoded_dataset.rename_column("emotion", "labels")

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset1["train"],
    eval_dataset=encoded_dataset1["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

In [32]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,2.079500,2.078283,0.100694
2,2.070300,2.074346,0.118056
3,2.062000,2.063207,0.159722
4,2.044400,2.043860,0.190972
5,2.003100,1.976246,0.277778
6,1.963200,1.842074,0.357639
7,1.824900,1.707176,0.388889
8,1.673300,1.572880,0.381944
9,1.545200,1.482600,0.420139
10,1.447900,1.421389,0.423611


TrainOutput(global_step=540, training_loss=1.3415313703042488, metrics={'train_runtime': 3197.679, 'train_samples_per_second': 10.808, 'train_steps_per_second': 0.169, 'total_flos': 2.09485473841152e+18, 'train_loss': 1.3415313703042488, 'epoch': 30.0})

In [33]:
trainer.evaluate()

{'eval_loss': 0.973603367805481,
 'eval_accuracy': 0.6354166666666666,
 'eval_runtime': 9.7578,
 'eval_samples_per_second': 29.515,
 'eval_steps_per_second': 1.845,
 'epoch': 30.0}

In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/1.18G [00:00<?, ?B/s]

Upload file runs/Aug19_13-01-17_8fef4eb7181c/events.out.tfevents.1692450446.8fef4eb7181c.11568.0:   0%|       …

Upload file runs/Aug19_13-01-17_8fef4eb7181c/events.out.tfevents.1692453653.8fef4eb7181c.11568.1:   0%|       …